In [3]:
import os
os.getenv("LD_LIBRARY_PATH")

':/home/annazan/miniconda3/envs/fair38/lib/'

In [4]:
!echo $LD_LIBRARY_PATH
!source ~/.bashrc
!echo $LD_LIBRARY_PATH


/bin/bash: /home/annazan/miniconda3/envs/fair38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
:/home/annazan/miniconda3/envs/fair38/lib/


/bin/bash: /home/annazan/miniconda3/envs/fair38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/annazan/miniconda3/envs/fair38/lib/libtinfo.so.6: no version information available (required by /bin/bash)
:/home/annazan/miniconda3/envs/fair38/lib/


In [5]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


2024-02-13 21:40:57.706349: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-13 21:40:57.706385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (buifoot): /proc/driver/nvidia/version does not exist


In [6]:
import os
import sys
print(os.getcwd())
os.environ.update(os.environ)
        # Add a new environment variable to the operating system
os.environ["RAMP_HOME"] = os.getcwd()
# Print the environment variables to verify that the new variable was added
print(os.environ["RAMP_HOME"])
sys.path.append('../')
sys.path.append('../ramp-code/')
sys.path.append('ramp-code')


/home/annazan/fAIr-utilities
/home/annazan/fAIr-utilities


In [7]:
import cv2

In [8]:
os.environ["SM_FRAMEWORK"] = "tf.keras"
import ramp.utils
import hot_fair_utilities
# base_path = f"{os.getcwd()}/ramp-data/sample_2"
# base_path = "/home/annazan/fAIr-utilities/ramp-data/test_data/1_Zanzibar"
# base_path = f"{os.getcwd()}/ramp-data/test_data/1_Zanzibar"
base_path = f'{os.getcwd()}/ramp-data/metric_data/model51_td364/'

Segmentation Models: using `tf.keras` framework.


In [9]:
# from hot_fair_utilities import preprocess
# model_input_image_path = f"{base_path}/input"
# preprocess_output=f"{base_path}/preprocessed"
# preprocess(
#             input_path = model_input_image_path,
#             output_path = preprocess_output,
#             rasterize=True,
#             rasterize_options=["binary"],
#             georeference_images=True,
#         )

In [10]:
from hot_fair_utilities import train

In [11]:
preprocess_output=f"{base_path}"

In [12]:
train_output = f"{base_path}train"
final_accuracy, final_model_path = train(
    input_path=preprocess_output,
    output_path=train_output,
    epoch_size=2,
    batch_size=2,
    model="ramp",
    model_home=os.environ["RAMP_HOME"]
)

variables are /home/annazan/fAIr-utilities/ramp-data/metric_data/model51_td364/
 and /home/annazan/fAIr-utilities/ramp-data/metric_data/model51_td364/train
Starting to prepare data for training
ramp home is /home/annazan/fAIr-utilities
python home is None
variables are: src /home/annazan/fAIr-utilities/ramp-data/metric_data/model51_td364/
 and dst:/home/annazan/fAIr-utilities/ramp-data/metric_data/model51_td364/train


Writing /home/annazan/fAIr-utilities/ramp-data/metric_data/model51_td364/train/fair_split_train.csv
Writing /home/annazan/fAIr-utilities/ramp-data/metric_data/model51_td364/train/fair_split_val.csv


done split
Data is ready for training


NameError: name 'edited_constructors' is not defined

[here a tf file is created (weights + structure)]

In [ ]:
print(final_accuracy,final_model_path)

In [ ]:
prediction_output = f"{os.getcwd()}/outputs/model51_td364/prediction"

In [ ]:


from hot_fair_utilities import predict


print(f"\n**\n** prediction output {prediction_output}")
print(f"\n**\n** prediction input {base_path}prediction/input")
predict(
    checkpoint_path=final_model_path,
    input_path=f"{base_path}prediction/input",
    prediction_path=prediction_output,
)

In [ ]:
from hot_fair_utilities import polygonize
geojson_output= f"{prediction_output}/prediction.geojson"
polygonize(
    input_path=prediction_output, 
    output_path=geojson_output,
    remove_inputs = True,
)